# EXTRACT EVENTS EMS

In [1]:
import feedparser as fp
import pandas as pd
import ssl

An error may be raised when running `fp.parse`. I was able to solve changing the module function as explained here: https://github.com/kurtmckee/feedparser/pull/131

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context
feeds=fp.parse('https://emergency.copernicus.eu/mapping/activations-rapid/feed')

In [3]:
print(len(feeds['entries']))

381


In [4]:
from bs4 import BeautifulSoup
print(feeds['entries'][0]['summary'])

<br />
				<b>Type of Event:</b> Flood (Riverine flood);<br />
				<b>Date/Time of Event (UTC):</b> 2020-01-23 10:00:00;<br />
				<b>Activation Date/Time (UTC):</b> 2020-01-23 15:52:00;<br />
				<b>Dedicated Mechanism:</b> Copernicus EMS (RM);<br />
				<b>Affected Country:</b> Spain;<br />
				<b>Affected Area:</b> ;<br />
				<b>Event Description:</b><br /><p>Due to Storm Gloria, Catalonia is suffering extended damages from flooding and strong winds. In the coming hours, it&#039;s expected that river levels will rise in the area of Girona, affecting people and infrastructure, especially in Ter and Tordera river basins.<br />
Copernicus EMS Rapid Mapping is requested to monitor the extent of floods on the 23rd and 24th of January 2020. Grading of damages arel also requested.</p>
<br />
				<br />
			<b>Details, maps:</b><br />&#9656;  <a target="_blank" title="[EMSR422] Flood in Girona, Catalonia, Spain" alt="[EMSR422] Flood in Girona, Catalonia, Spain" href="https://emergency.coper

In [5]:
list_of_events=list()
for entry in feeds['entries']:
    #print(entry.keys())
    #print(entry['summary_detail']['value'])
    #we loop throught the whole list of activations
    
    #soup munch data in html format
    soup = BeautifulSoup(entry['summary_detail']['value'], 'html.parser')
    #some sanitation to get the 1st 6 fields
    data=soup.prettify().replace('<br/>','').split('<p>')
    #replace('\n','').replace('<div>','').replace('</div>','').split('<br/>')
    first_half=data[0].replace('<b>','').replace('</b>','').split(';')
    first_half_splitted=[first_half_sp.replace('\n','').lstrip() for first_half_sp in first_half]
    activation=dict()
    for item in first_half_splitted[:-1]:
        kv=item.split(':')
        #need a check because sometimes ; is added to locations
        if len(kv)<2 :
            pass
        else:
            k=kv[0]
            v=kv[1]
            activation[k]=v
        
        
    #now second part with link and imgs 
    #print('<p>'+data[1])
    soup=BeautifulSoup('<p>'+data[1], 'html.parser')
    #links contains details and the feed url
    links=soup.find_all('a',href=True)
    #img is the marker for maps
    marker=soup.find_all('img')
    activation['Event Description']=soup.find('p').get_text().replace('\n','')
    if len(links)>0: 
        activation['Details, maps:']=links[0]['href']
        activation['Activation Feed (GeoRSS)']=links[1]['href']
        activation['Activation Location (preview)']=marker[0]['src']

    else:
        activation['Details, maps:']=''
        activation['Activation Feed (GeoRSS)']=''
        activation['Activation Location (preview)']=''
    
    if 'flood' in activation['Type of Event'].lower(): list_of_events.append(activation)

In [6]:
data = pd.DataFrame.from_records(list_of_events)
data.head()

,Type of Event,Date/Time of Event (UTC),Activation Date/Time (UTC),Dedicated Mechanism,Affected Country,Affected Area,Event Description,"Details, maps:",Activation Feed (GeoRSS),Activation Location (preview)
0,Flood (Riverine flood),2020-01-23 10,2020-01-23 15,Copernicus EMS (RM),Spain,,"Due to Storm Gloria, Catalonia is suffering e...",https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
1,Flood (Riverine flood),2020-01-22 18,2020-01-22 20,Copernicus EMS (RM),France,,Heavy rainfall is occurring over the Pyrénées...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
2,Flood (Riverine flood),2020-01-13 00,2020-01-13 17,Copernicus EMS (RM),Iran,,Days of heavy rain from 09 January 2020 cause...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
3,Flood (Riverine flood),2019-12-19 01,2019-12-23 14,Copernicus EMS (RM),Portugal,,"Starting from 19/12/2019, severe storms affec...",https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
4,Flood (Flash flood),2019-12-13 20,2019-12-13 20,Copernicus EMS (RM),France,,Another round of bad weather swept France’s A...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...


In [7]:
data.rename(columns={'Activation Feed (GeoRSS)': 'id_EMS', 'Affected Area': 'area', 'Affected Country': 'countries', 'Date/Time of Event (UTC)': 'start_date', 'Type of Event': 'type'}, inplace=True)
data = data.drop(columns=['Activation Location (preview)', 'Dedicated Mechanism','Details, maps:', 'Event Description', 'Activation Date/Time (UTC)'])
data.head()

,type,start_date,countries,area,id_EMS
0,Flood (Riverine flood),2020-01-23 10,Spain,,https://emergency.copernicus.eu/mapping/list-o...
1,Flood (Riverine flood),2020-01-22 18,France,,https://emergency.copernicus.eu/mapping/list-o...
2,Flood (Riverine flood),2020-01-13 00,Iran,,https://emergency.copernicus.eu/mapping/list-o...
3,Flood (Riverine flood),2019-12-19 01,Portugal,,https://emergency.copernicus.eu/mapping/list-o...
4,Flood (Flash flood),2019-12-13 20,France,,https://emergency.copernicus.eu/mapping/list-o...


In [8]:
data['start_date'] = pd.to_datetime(data['start_date'], errors='coerce')

In [9]:
data = data.sort_values(by='start_date')
data.index = data.pop('start_date')

In [10]:
data.index = data.index.normalize()

In [11]:
data['in_EMS'] = 1

In [12]:
data.head()

,type,countries,area,id_EMS,in_EMS
start_date,,,,,
2012-07-07,Flood,Sweden,Kalmar Highland and Jonkoping Highland,https://emergency.copernicus.eu/mapping/list-o...,1
2012-08-06,Flood,Philippines,"Olongapo, Malolos, Manila, Pampanga",https://emergency.copernicus.eu/mapping/list-o...,1
2012-08-24,Flood,Niger,"Niamey, Dosso, Agades",https://emergency.copernicus.eu/mapping/list-o...,1
2012-09-06,Flood,Cameroon,North and Extreme North,https://emergency.copernicus.eu/mapping/list-o...,1
2012-11-05,Flood,Slovenia,East and north-east of the Country,https://emergency.copernicus.eu/mapping/list-o...,1


In [13]:
outfile = "./data/EMS/database.csv"
data.to_csv(outfile, sep=',')